In [ ]:
%pip install crewai 'crewai[tools]' langchain_community spider-client firecrawl-py --quiet



In [2]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, LLM
import os

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
SERPER_API_KEY = os.getenv('SERPER_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
SPIDER_API_KEY = os.getenv('SPIDER_API_KEY')
JINAAI_API_KEY = os.getenv('JINAAI_API_KEY')

In [3]:
from crewai_tools import (
  TXTSearchTool,
  ScrapeWebsiteTool,
  SerperDevTool,
  SpiderTool,
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
spider_tool = SpiderTool(api_key=SPIDER_API_KEY)
json_tool = TXTSearchTool(
    description="Searches a TXT file for relevant information based on the provided query.",
    json_path="health_data.txt",
)

In [4]:
health_agent = Agent(
    role="Smart Health Context Analyzer",
    goal="Check TXT file for relevant information about the query",
    tools = [json_tool],
    verbose=True,
    backstory=(
        "You analyze patient data and TXT records"
        "to determine if any relevant information exists for the query."
        "However, only do so if the query is user-specific"
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [5]:
search_agent = Agent(
    role="Senior Web Search Researcher",
    goal="Search trusted medical websites for relevant information for the query and return the top 4 sources",
    tools = [search_tool],
    verbose=True,
    backstory=(
        "You are skilled at finding reliable medical information from trusted websites that end with .gov, .edu, and .org."
        "An expert web researcher that uses the web extremely well"
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [6]:
scrape_agent = Agent(
    role="Amazing Web Content Collecter",
    goal="You're a researcher that is tasked with researching trusted medical websites and their content" ,
    tools = [scrape_tool],
    verbose=True,
    backstory=(
        "You are very skilled at scraping the main content from websites"
        "You are also very skilled at generating a structured report with clear citations"
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [7]:
synthesis_agent = Agent(
    role="Professional Medical Information Synthesizer",
    goal="Combine results from web searches, health data, and other sources into a coherent response to the query",
    tools = [],
    verbose=True,
    backstory=(
        "You synthesize data from multiple sources to create clear and concise answers for the user."
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [8]:
health_context_task = Task(
    description=(
        "Check if the query EXPLICITLY references the user's health (e.g., contains 'my condition,' 'for me')"
        "If NO, then return '' and ignore the rest of task. If YES, proceed to the following steps."
        "Only procceed to checking TXT file if YES to the check above."
        "Provide relevant medical context from the TXT file to help answer the {query}"
        "Your tasks:"
        "1. Review data in the TXT file containing user's health data."
        "2. Extract relevant health information"
        "Ensure there's no hallucination and include citations for each piece of information"
    ),
    expected_output=(
        "A structured health context report."
    ),
    agent=health_agent,
    async_execution=True
)

In [9]:
web_search_task = Task(
    description=(
        "Search exactly 4 trusted medical websites for information about: {query}\n"
        "Your task:"
        "1. Search ONLY trusted medical websites using: (site:.gov OR site:.edu OR site:.org)\n"
        "2. Limit to exactly 4 top sources\n"
        "3. Return a list called website_urls containing the 4 top source links"
    ),
    expected_output=(
        "A list website_urls that contains the 4 medical sources with the format website_urls = ['link 1', 'link 2', 'link 3', 'link 4']"
    ),
    agent=search_agent,
    async_execution=True
)

In [10]:
scrape_task = Task(
    description=(
        "Scrape the given URLs using the scrape tool.\n"
        "Return the extracted readable content and metadata."
    ),
    expected_output=(
        "A structured report from exactly 4 medical sources with clear citations."
    ),
    context=[web_search_task],
    agent=scrape_agent,
)

In [11]:
synthesis_task = Task(
    description=(
        "Create a comprehensive answer for: {query}"
        "Using:"
        "Health Context (if available)"
        "Web Search Results"
        "List all sources at the end"
        "Include a brief medical disclaimer"
        "Keep the response clear and well-structured."
    ),
    expected_output=(
        "A clear medical response with proper organization and citations with links."

    ),
    context=[health_context_task, scrape_task],
    agent=synthesis_agent
)

In [12]:
health_crew = Crew(
    agents=[health_agent,
            search_agent,
            scrape_agent,
            synthesis_agent,
    ],
    tasks=[health_context_task,
           web_search_task,
           scrape_task,
           synthesis_task,
    ],
    verbose=True
)

In [13]:
input = {'query':'What is an astrocytoma?'}
result = health_crew.kickoff(inputs=input)
from IPython.display import Markdown
Markdown(str(result))

# Agent: Smart Health Context Analyzer
## Task: Check if the query EXPLICITLY references the user's health (e.g., contains 'my condition,' 'for me')If NO, then return '' and ignore the rest of task. If YES, proceed to the following steps.Only procceed to checking TXT file if YES to the check above.Provide relevant medical context from the TXT file to help answer the What is an astrocytoma?Your tasks:1. Review data in the TXT file containing user's health data.2. Extract relevant health informationEnsure there's no hallucination and include citations for each piece of information
# Agent: Senior Web Search Researcher
## Task: Search exactly 4 trusted medical websites for information about: What is an astrocytoma?
Your task:1. Search ONLY trusted medical websites using: (site:.gov OR site:.edu OR site:.org)
2. Limit to exactly 4 top sources
3. Return a list called website_urls containing the 4 top source links


# Agent: Smart Health Context Analyzer
## Final Answer: 





# Agent: Senio

An astrocytoma is a type of tumor that develops in the central nervous system, primarily affecting the brain and occasionally the spinal cord. These tumors originate from astrocytes, which are star-shaped glial cells that support nerve cells. Astrocytomas can be either benign (non-cancerous) or malignant (cancerous) and are classified into four grades by the World Health Organization, with Grade 1 being the least aggressive and Grade 4 being the most aggressive, known as glioblastoma.

The classification of astrocytomas has evolved to include genetic factors, such as IDH mutations, which play a crucial role in diagnosis and treatment planning. Treatment typically involves a combination of surgery, radiation therapy, and chemotherapy. Newer therapies, including genetic and immunological approaches, are being explored in clinical trials to improve outcomes.

The prognosis for individuals with astrocytoma varies significantly based on the tumor's grade, genetic characteristics, and other factors such as the patient's age and overall health. Regular monitoring through MRI scans is often necessary to track the tumor's progression and response to treatment.

For more detailed information, you can refer to the following sources:
- [Cleveland Clinic](https://my.clevelandclinic.org/health/diseases/17863-astrocytoma)
- [MD Anderson Cancer Center](https://www.mdanderson.org/cancerwise/what-is-astrocytoma--and-how-is-it-different-from-glioblastoma.h00-159694389.html)
- [American Association of Neurological Surgeons (AANS)](https://www.aans.org/patients/conditions-treatments/astrocytoma-tumors/)
- [American Brain Tumor Association (ABTA)](https://www.abta.org/tumor_types/astrocytoma/)

**Medical Disclaimer**: This response is for informational purposes only and should not be considered medical advice. Please consult a healthcare professional for personalized medical advice.